In [1]:
import ee
import geemap

In [2]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [3]:
NLCD2016 = ee.Image('USGS/NLCD/NLCD2016').select('landcover')
Map.addLayer(NLCD2016, {}, 'NLCD 2016')

In [4]:
region = ee.Geometry.Rectangle([-84.42291, 30.55397, -84.14871, 30.08362])

In [5]:
metadata = NLCD2016.clip(region)
region2 = metadata.geometry()
Map.addLayer(region2,{},'NLCD')
Map

Map(bottom=1826.0, center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(Togg…

In [6]:
metadata.get('2016on_bas').getInfo()
metadata

In [7]:
dataset = ee.ImageCollection('USDA/NAIP/DOQQ').filter(ee.Filter.date('2017-01-01', '2018-12-31')).filterBounds(region)

In [8]:
new=dataset.median()

In [9]:
Map.addLayer(new,{},'NAIP')
Map

Map(bottom=787.0, center=[0, -50.09765625000001], controls=(WidgetControl(options=['position'], widget=HBox(ch…

In [10]:
raw_class_values = metadata.get('landcover_class_values').getInfo()
print(raw_class_values)

[11, 12, 21, 22, 23, 24, 31, 41, 42, 43, 51, 52, 71, 72, 73, 74, 81, 82, 90, 95]


In [12]:
n_classes = len(raw_class_values)
new_class_values = list(range(0, n_classes))
new_class_values

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [13]:
class_palette = metadata.get('landcover_class_palette').getInfo()
print(class_palette)

['476ba1', 'd1defa', 'decaca', 'd99482', 'ee0000', 'ab0000', 'b3aea3', '68ab63', '1c6330', 'b5ca8f', 'a68c30', 'ccba7d', 'e3e3c2', 'caca78', '99c247', '78ae94', 'dcd93d', 'ab7028', 'bad9eb', '70a3ba']


In [14]:
nlcd = metadata.remap(raw_class_values, new_class_values).select(['remapped'], ['landcover'])
nlcd = nlcd.set('landcover_class_values', new_class_values)
nlcd = nlcd.set('landcover_class_palette', class_palette)

In [15]:
Map.addLayer(nlcd, {}, 'NLCD')
Map

Map(bottom=787.0, center=[0, -50.09765625000001], controls=(WidgetControl(options=['position'], widget=HBox(ch…

In [16]:
points = metadata.sample(**{
    'region': dataset.geometry(),
    'scale': 30,
    'numPixels': 190000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(points, {}, 'training', False)

In [17]:
print(points.size().getInfo())

69722


In [18]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [-84.14894278538817, 30.34757834187164]}, 'id': '0', 'properties': {'landcover': 52}}


In [45]:
bands = bands = ['R', 'G', 'B', 'N']

# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
sample = new.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Adds a column of deterministic pseudorandom numbers. 
sample = sample.randomColumn()

split = 0.7

training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

In [48]:
classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)

In [49]:
result = new.select(bands).classify(classifier)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classfied')
Map

Map(bottom=108083.0, center=[30.41078179084589, -84.14772033691408], controls=(WidgetControl(options=['positio…

In [50]:
class_values = nlcd.get('landcover_class_values').getInfo()
print(class_values)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [51]:
class_palette = nlcd.get('landcover_class_palette').getInfo()
print(class_palette)

['476ba1', 'd1defa', 'decaca', 'd99482', 'ee0000', 'ab0000', 'b3aea3', '68ab63', '1c6330', 'b5ca8f', 'a68c30', 'ccba7d', 'e3e3c2', 'caca78', '99c247', '78ae94', 'dcd93d', 'ab7028', 'bad9eb', '70a3ba']


In [53]:
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [54]:
Map.addLayer(landcover, {}, 'Land cover')
Map

Map(bottom=108083.0, center=[30.41078179084589, -84.14772033691408], controls=(WidgetControl(options=['positio…

In [55]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [56]:
Map.add_legend(builtin_legend='NLCD')
Map

Map(bottom=108040.0, center=[30.461695615612108, -84.14634704589845], controls=(WidgetControl(options=['positi…

In [57]:
train_accuracy = classifier.confusionMatrix()

In [58]:
train_accuracy.getInfo()

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [59]:
train_accuracy.accuracy().getInfo()

0.8271197556926483

In [60]:
train_accuracy.kappa().getInfo()

0.7720254964772622

In [61]:
train_accuracy.producersAccuracy().getInfo()

[[0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.9666395443449959],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.814801041279286],
 [0.8471953578336557],
 [0.8795761078998073],
 [0.8541666666666666],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.782608695652174],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.7236842105263158],
 [0.9689524698077022],
 [0.49014503532911863],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.649536647009267],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.8384074941451991],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.7870778267254038],
 [0.7768361581920904],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.6088663711209626],
 [0],
 [0],
 [0],
 [0],
 [0.8913649025069638]]

In [62]:
train_accuracy.consumersAccuracy().getInfo()

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0.9406175771971497,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0.8269484808454426,
  0.8961636828644501,
  0.9084577114427861,
  0.9169329073482428,
  0,
  0,
  0,
  0,
  0,
  0,
  0.9545454545454546,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0.9565217391304348,
  0.7537903225806452,
  0.8984321745057942,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0.927797833935018,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0.903063063063063,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0.9453262786596119,
  0.9450171821305842,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0.9357601713062098,
  0,
  0,
  0,
  0,
  0.9552238805970149]]

In [63]:
validated = validation.classify(classifier)

In [64]:
validated.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '2_0',
 'properties': {'B': 99,
  'G': 113,
  'N': 227,
  'R': 87,
  'classification': 42,
  'landcover': 90,
  'random': 0.9791335768231186}}

In [65]:
test_accuracy = validated.errorMatrix('landcover', 'classification')

In [66]:
test_accuracy.getInfo()

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [68]:
test_accuracy.accuracy().getInfo()

0.43925278295351394

In [69]:
test_accuracy.kappa().getInfo()

0.2425117008973367

In [70]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.7683823529411765],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.2747018739352641],
 [0.20441347270615565],
 [0.22748815165876776],
 [0.2348993288590604],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.022222222222222223],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.014925373134328358],
 [0.7454721081864284],
 [0.06881533101045297],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.07955596669750231],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.32300542215336947],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.15355805243445692],
 [0.15454545454545454],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.15474408859622868],
 [0],
 [0],
 [0],
 [0],
 [0.5268817204301075]]

In [71]:
import csv
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
training_csv = os.path.join(out_dir, 'train_accuracy_NAIP.csv')
testing_csv = os.path.join(out_dir, 'test_accuracy_NAIp.csv')

with open(training_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(train_accuracy.getInfo())
    
with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(test_accuracy.getInfo())

In [72]:
landcover = landcover.remap(new_class_values, raw_class_values).select(['remapped'], ['classification'])

In [73]:
landcover = landcover.set('classification_class_values', raw_class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [74]:
Map.addLayer(landcover, {}, 'Final land cover')
Map

Map(bottom=431460.0, center=[30.424696928720373, -84.20557022094728], controls=(WidgetControl(options=['positi…

In [75]:
import os
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'landcover.tif')

In [79]:
geemap.ee_export_image_to_drive(landcover, description='landcoverNAIP2020publication1', folder='export3',scale=1)

Exporting landcoverNAIP2020publication1 ...
